In [ ]:
import pandas as pd
import numpy as np

In [ ]:
constraints = pd.read_xml(r'/mnt/c/users/ty90rize/repos/traffic-simulation/04_sumo_files/edge-data-file.xml',xpath='interval/edge')
constraints

### Load sumo network

In [ ]:
## get sumo network
import sys
sys.path.append(r'/mnt/c/users/ty90rize/Sumo/tools') # path to sumolib
import sumolib

In [ ]:
path_osm = '../roadmatching/oktober_data/big_map/osm_sumo_data/2023-11-27-15-41-55/osm.net.xml/osm_final_2024-02-25.net.xml'

In [ ]:
net_osm = sumolib.net.readNet(path_osm)

## remove edges with inconsistencies

In [ ]:
def save_df_to_xml(df, filename):
    import xml.etree.cElementTree as ET

    ids_of_interest = [1]

    root = ET.Element("data")
    intervals = dict()
    for idx, elem in enumerate(ids_of_interest):
        interval = ET.SubElement(root, "interval", id=str(elem), begin=str(idx*3600), end=str(idx*3600+3599))
        print(interval)
        intervals[str(elem)] = interval
        
    for idx, row in df.iterrows():
        interval = intervals[str(ids_of_interest[0])]
        ET.SubElement(interval, "edge", id = row['id'], entered = str(row['entered']))

    tree = ET.ElementTree(root)
    ET.indent(tree, space = "\t", level = 0)
    tree.write(filename)

In [ ]:
iscompletes = []
differences_in = []
differences_out = []
edgecounts = []
edgeids = []
nooutgoing = []
noincoming = []

for idx, row in constraints.iterrows():
    osmid = row['id']
    edge = net_osm.getEdge(osmid)
    incomings = [e.getID() for e in edge.getIncoming()]
    outgoings = [e.getID() for e in edge.getOutgoing()]
    if outgoings == []:
        nooutgoing.append(True)
    else:
        nooutgoing.append(False)
    if incomings == []:
        noincoming.append(True)
    else:
        noincoming.append(False)
    summe = 0
    summe_out = 0
    problems = -1
    for edgeid in incomings:
        try:
            summe += constraints[constraints['id'] == edgeid]['entered'].iloc[0]
        except IndexError:
            problems += 1
    for edgeid in outgoings:
        try:
            summe_out += constraints[constraints['id'] == edgeid]['entered'].iloc[0]
        except IndexError:
            problems += 1
    if problems == -1:
        iscompletes.append(True)
    else:
        iscompletes.append(False)
    edgecounts.append(row['entered'])
    edgeids.append(edge.getID())
    differences_in.append(summe - row['entered'])
    differences_out.append(row['entered'] - summe_out)

In [ ]:
stat_df = pd.DataFrame({'ID':edgeids, 'Counts':edgecounts, 'DiffIn':differences_in, 
                        'DiffOut':differences_out, 'iscomplete':iscompletes,
                       'noincoming':noincoming, 'nooutgoing':nooutgoing})
stat_df

In [ ]:
stat_df['max_abs_diff'] = stat_df.apply(lambda x: 
                                        max(np.abs(x['DiffIn']), np.abs(x['DiffOut'])),
                                       axis = 1)
stat_df['max_rel_diff'] = stat_df['max_abs_diff'] / stat_df['Counts']

In [ ]:
stat_df[(stat_df['max_rel_diff']>0.35) & (stat_df['iscomplete']==True) & (stat_df['max_abs_diff'] > 100)]

In [ ]:
problematic_edge_ids = stat_df[(stat_df['max_rel_diff']>0.35) & 
                               (stat_df['iscomplete']==True) & 
                               (stat_df['max_abs_diff'] > 100) & 
                              (stat_df['noincoming']==False) &
                              (stat_df['nooutgoing']==False)]['ID'].tolist()

In [ ]:
constraints = constraints[~constraints['id'].isin(problematic_edge_ids)]

In [ ]:
save_df_to_xml(constraints, "constraints_reduced_conflicts.xml")

## Manual corrections

In [ ]:
new_row = {'id':'E1', 'entered':2460}

In [ ]:
constraints = constraints.append(new_row, ignore_index = True)

In [ ]:
new_row = {'id':'E2', 'entered':2460}

In [ ]:
constraints = constraints.append(new_row, ignore_index = True)

In [ ]:
save_df_to_xml(constraints, "constraints_final.xml")